<a href="https://colab.research.google.com/github/BrahianCarrera/KaggleCompetition/blob/main/02-preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
# Carga el archivo kaggle.json
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai4eng-20241
!unzip udea-ai4eng-20241.zip

Saving kaggle.json to kaggle (1).json
udea-ai4eng-20241.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  udea-ai4eng-20241.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission_example.csv  
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [107]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import re

In [136]:
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')
df.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


In [ ]:
category_cols = [i for i in df.columns if not i in df._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(df[c].dropna()))


# Analizar datos


# Eliminar Columnas

Se observo que la columna Estudio Privado de Libertad no soloe esta mal factorizada sino que todos los resultados son el mismo: No. Por tanto podemos obviar esta informacion y borrarla directamente del Dataframe.

Despues de este paso usaremos el df resultante como la base de las demas copias

In [137]:
df.isnull().sum()

ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PRIVADO_LIBERTAD                 0
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_TIENEINTERNET.1              26629
FAMI_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                    0
dtype: int64

In [138]:
df = df.drop(labels =['ESTU_PRIVADO_LIBERTAD','FAMI_TIENEINTERNET.1'],axis= 1)        #Normal, Tratando de usar la mayor cantidad de informacion posible

# Renombrar las columnas

In [139]:
print(df.columns)
df.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE','RENDIMIENTO_GLOBAL']

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
       'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


# Convertir a binario


In [140]:
def convertir_num(valor):
    if valor == 'Si':
        return 1
    elif valor == 'No':
        return 0
    else:
        return valor  # Si no es 'Si' ni 'No', dejar el valor original

# Aplicar la función a todas las columnas del DataFrame
df = df.applymap(convertir_num)


In [141]:
frecuencia_valores = df_replaced['ESTU_PRGM_ACADEMICO'].value_counts()
frecuencia_valores

ESTU_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
LICENCIATURA EN ESPAÑOL Y FILOLOGÍA                           1
ADMINISTRACION EN NEGOCIOS INTERNACIONALES                    1
ADMINISTRACION DE COMERCIO EXTERIOR                           1
LICENCIATURA EN EDUCACI¿N F¿SICA RECREACI¿N Y DEPORTES        1
PROFESIONAL EN INVESTIGACIÓN CRIMINAL                         1
Name: count, Length: 948, dtype: int64

# Limpiar datos faltantes


In [142]:
df.dropna(inplace = True)
#df.fillNa(0)
#df.fillNa(promedios)



# Convertir Datos Categoricos

Convertir las columnas de tipo objeto que ya identificamos como variables categoricas al tipo categorico




In [143]:
df["RENDIMIENTO_GLOBAL"].value_counts()

valores_rendimiento = {'alto': 3,'medio-alto': 2.5,'medio-bajo':1.5, 'bajo': 1}
df= df.replace(valores_rendimiento)

In [144]:
Columnas_categoricas = ["ESTRATO","VALOR_MATRICULA","PROGRAMA","DEPARTAMENTO",]

df[Columnas_categoricas] = df[Columnas_categoricas].astype("category")

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 628896 entries, 0 to 692499
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   ID                  628896 non-null  int64   
 1   PERIODO             628896 non-null  int64   
 2   PROGRAMA            628896 non-null  category
 3   DEPARTAMENTO        628896 non-null  category
 4   VALOR_MATRICULA     628896 non-null  category
 5   HORAS_TRABAJA       628896 non-null  object  
 6   ESTRATO             628896 non-null  category
 7   TIENE_INTERNET      628896 non-null  float64 
 8   EDUCACION_PADRE     628896 non-null  object  
 9   TIENE_LAVADORA      628896 non-null  float64 
 10  TIENE_AUTO          628896 non-null  float64 
 11  PAGA_MATRICULA      628896 non-null  float64 
 12  TIENE_COMPUTADOR    628896 non-null  float64 
 13  EDUCACION_MADRE     628896 non-null  object  
 14  RENDIMIENTO_GLOBAL  628896 non-null  float64 
dtypes: category(4), float6

# Normalizar rangos


In [145]:
def normalizar(valor):
  max = 7
  min = 0.5
  if (valor == "Entre 1 millón y menos de 2.5 millones"):
    return((1+2.5)/2)

  elif(valor == "Entre 2.5 millones y menos de 4 millones"):
    return((2.5+4)/2)

  elif(valor == "Menos de 500 mil"):
    return(0.5)

  elif(valor == "Entre 500 mil y menos de 1 millón"):
    return((0.5+1)/2)

  elif(valor == "Entre 4 millones y menos de 5.5 millones"):
    return((4+5.5)/2)

  elif(valor == "Entre 5.5 millones y menos de 7 millones"):
    return((5.5+7)/2)

  elif(valor == "No pagó matrícula"):
    return(0)

df["VALOR_MATRICULA"] = df["VALOR_MATRICULA"].apply(normalizar)


In [ ]:
scaler = MinMaxScaler()
df['VALOR_MATRICULA'] = scaler.fit_transform(df[['VALOR_MATRICULA']])
df

# **Convertir a One Hot las columnas pertinentes**
Convertir el rango de string a tipo numerico


In [ ]:
codificador = OneHotEncoder()

Columnas_OneHot=['VALOR_MATRICULA']
codificacion = codificador.fit_transform(df[Columnas_OneHot])

df_codificado = pd.DataFrame(codificacion.toarray(), columns=codificador.get_feature_names_out(Columnas_OneHot))

df = pd.concat([df,df_codificado], axis = 1 )

df.drop(labels =['VALOR_MATRICULA_nan','VALOR_MATRICULA'], axis = "columns", inplace =True)


In [ ]:

df2 = df.copy()                                               #Dropna(), borrando todo lo que no tenga Na
df3 = df.copy()                                               #fillna() rellenando con valores lo que sea Na y se pueda rellenar
df = df.sort_index(axis=0)

In [ ]:
## Cosas por hacer
##Hacer estadistica
##ver los videos del profe
##Meter los datos de programa que tengan poca frecuencia para que no resulten tantas casillas a la hora de hacer one hot



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar el tamaño de la figura
plt.figure(figsize=(20, 13))

# Recorrer cada variable categórica y crear subplots
for i, c in enumerate(["VALOR_MATRICULA", "HORAS_TRABAJA", "TIENE_LAVADORA", "TIENE_COMPUTADOR"]):
    # Primer subplot: distribución de SalePrice para cada valor único de la variable categórica
    plt.subplot(2, 4, i + 1)
    k = df[[c, "RENDIMIENTO_GLOBAL"]].dropna()  # Filtrar filas con valores no nulos en las columnas c y SalePrice
    for v in df[c].dropna().unique():  # Recorrer cada valor único de la variable categórica
        sns.histplot(k.RENDIMIENTO_GLOBAL[k[c] == v], label=v, kde=True)  # Graficar la distribución de SalePrice para el valor v de la variable c
        plt.title(c)  # Establecer el título como el nombre de la variable categórica
    plt.yticks([])  # Ocultar las etiquetas en el eje y
    plt.legend()  # Mostrar la leyenda

    # Segundo subplot: gráfico de barras para contar la frecuencia de cada valor en la variable categórica
    plt.subplot(2, 4, i + 5)
    vc = k[c].value_counts()  # Contar la frecuencia de cada valor en la variable categórica
    sns.barplot(vc.values)  # Graficar el gráfico de barras
    plt.xticks(range(len(vc)), vc.index, rotation="vertical")  # Establecer las etiquetas del eje x y rotarlas verticalmente

plt.tight_layout()  # Ajustar el diseño de los subplots para evitar superposiciones
plt.show()  # Mostrar el gráfico
